In [1]:
import pandas as pd
import numpy as np
import os
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss,roc_auc_score
from copy import deepcopy
import io
import copy
from contextlib import redirect_stdout
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score

/Users/boss/anaconda3/lib/python3.7/site-packages/lightgbm/__init__.py:46: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [2]:
train_path = './final/train_final.csv'
test_path = './final/test_final.csv'

In [3]:
dftrain = pd.read_csv(train_path)
dftest = pd.read_csv(test_path)
label = 'loan_status'

In [59]:
dftrain[label].value_counts()

1    39788
0    10212
Name: loan_status, dtype: int64

In [60]:
dftest[label].value_counts()

1    40226
0     9774
Name: loan_status, dtype: int64

In [4]:
feature = list(dftrain.columns)
feature.remove('loan_status')

In [5]:
x_train, x_valid, y_train, y_valid = train_test_split(dftrain[feature], dftrain[label], test_size = 0.2, random_state = 42)

In [6]:
dftrain.shape

(50000, 146)

In [7]:
dftest.shape

(50000, 146)

In [30]:
def eval_model(model, test_data):
    train_res = list(model.best_score.values())[0]
    print(f'训练集',train_res)
    ypred = model.predict(test_data[feature], num_iteration=bst.best_iteration)
    label_pred = (ypred>0.5).astype(int)
    score = accuracy_score(test_data[label], label_pred)
    print('测试集',score)
    fi = sorted(list(zip(model.feature_name(), model.feature_importance())),key=lambda x:x[1],reverse=True)
#     print('特征重要性',fi)
    return score

In [9]:
for f in feature:
    print(f)

continuous_annual_inc
continuous_annual_inc_joint
continuous_delinq_2yrs
continuous_dti
continuous_dti_joint
continuous_fico_range_high
continuous_fico_range_low
continuous_funded_amnt
continuous_funded_amnt_inv
continuous_inq_last_6mths
continuous_installment
continuous_int_rate
continuous_last_fico_range_high
continuous_last_fico_range_low
continuous_loan_amnt
continuous_mths_since_last_delinq
continuous_mths_since_last_major_derog
continuous_mths_since_last_record
continuous_open_acc
continuous_pub_rec
discrete_addr_state_1_one_hot
discrete_addr_state_2_one_hot
discrete_addr_state_3_one_hot
discrete_addr_state_4_one_hot
discrete_addr_state_5_one_hot
discrete_addr_state_6_one_hot
discrete_addr_state_7_one_hot
discrete_addr_state_8_one_hot
discrete_addr_state_9_one_hot
discrete_addr_state_10_one_hot
discrete_addr_state_11_one_hot
discrete_addr_state_12_one_hot
discrete_addr_state_13_one_hot
discrete_addr_state_14_one_hot
discrete_addr_state_15_one_hot
discrete_addr_state_16_one_hot
di

## 原始特征

In [10]:
train_data = lgb.Dataset(x_train, label=y_train)
valid_data = lgb.Dataset(x_valid, label=y_valid)
test_data = lgb.Dataset(dftest[feature], label=dftest[label])

In [11]:
param = { # 'num_leaves': 10, 
         'objective': 'binary',
    # 'learning_rate':0.01,
    'metric': 'binary'}
num_round = 1000
bst = lgb.train(param, train_data, num_round, valid_sets=[valid_data],early_stopping_rounds=50)

[1]	valid_0's binary_logloss: 0.448173
Training until validation scores don't improve for 50 rounds.
[2]	valid_0's binary_logloss: 0.405734
[3]	valid_0's binary_logloss: 0.372961
[4]	valid_0's binary_logloss: 0.34683
[5]	valid_0's binary_logloss: 0.325461
[6]	valid_0's binary_logloss: 0.30756
[7]	valid_0's binary_logloss: 0.292604
[8]	valid_0's binary_logloss: 0.279841
[9]	valid_0's binary_logloss: 0.268872
[10]	valid_0's binary_logloss: 0.259493
[11]	valid_0's binary_logloss: 0.251532
[12]	valid_0's binary_logloss: 0.244515
[13]	valid_0's binary_logloss: 0.23856
[14]	valid_0's binary_logloss: 0.233362
[15]	valid_0's binary_logloss: 0.2288
[16]	valid_0's binary_logloss: 0.224839
[17]	valid_0's binary_logloss: 0.221349
[18]	valid_0's binary_logloss: 0.218402
[19]	valid_0's binary_logloss: 0.215797
[20]	valid_0's binary_logloss: 0.213604
[21]	valid_0's binary_logloss: 0.211591
[22]	valid_0's binary_logloss: 0.209916
[23]	valid_0's binary_logloss: 0.208369
[24]	valid_0's binary_logloss: 0

In [12]:
origin_score = eval_model(bst, dftest)

训练集 {'binary_logloss': 0.19845525917626908}
测试集 0.91656
特征重要性 [('continuous_last_fico_range_high', 284), ('continuous_dti', 147), ('continuous_installment', 125), ('discrete_term_1_one_hot', 90), ('continuous_fico_range_high', 86), ('continuous_annual_inc', 74), ('continuous_mths_since_last_delinq', 72), ('continuous_funded_amnt', 70), ('discrete_emp_length_12_one_hot', 66), ('continuous_open_acc', 64), ('continuous_int_rate', 63), ('continuous_mths_since_last_record', 61), ('continuous_mths_since_last_major_derog', 36), ('continuous_funded_amnt_inv', 26), ('continuous_inq_last_6mths', 25), ('continuous_delinq_2yrs', 18), ('discrete_home_ownership_1_one_hot', 17), ('discrete_purpose_5_one_hot', 8), ('discrete_addr_state_14_one_hot', 7), ('discrete_addr_state_27_one_hot', 6), ('discrete_purpose_1_one_hot', 6), ('discrete_purpose_3_one_hot', 6), ('discrete_addr_state_3_one_hot', 5), ('discrete_addr_state_9_one_hot', 5), ('discrete_addr_state_15_one_hot', 4), ('discrete_addr_state_16_one_

## 衍生特征

In [13]:
ctn_features = [f for f in feature if f.split('_')[0]=='continuous']
disc_features = [f for f in feature if f.split('_')[0]=='discrete']

In [14]:
dftrain[ctn_features].describe()

,continuous_annual_inc,continuous_annual_inc_joint,continuous_delinq_2yrs,continuous_dti,continuous_dti_joint,continuous_fico_range_high,continuous_fico_range_low,continuous_funded_amnt,continuous_funded_amnt_inv,continuous_inq_last_6mths,continuous_installment,continuous_int_rate,continuous_last_fico_range_high,continuous_last_fico_range_low,continuous_loan_amnt,continuous_mths_since_last_delinq,continuous_mths_since_last_major_derog,continuous_mths_since_last_record,continuous_open_acc,continuous_pub_rec
count,5.000000e+04,220.000000,50000.00000,49999.000000,220.000000,50000.000000,50000.000000,50000.00000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.00000,26083.000000,15052.000000,9495.000000,50000.000000,50000.000000
mean,7.835886e+04,109248.802545,0.34738,19.080239,18.831864,698.404460,694.404300,14332.53650,14325.533000,0.620820,428.892109,11.986125,675.644860,658.428400,14332.53650,34.197485,44.214589,65.500790,11.944960,0.252820
std,9.783053e+04,52319.230212,0.91874,9.802002,7.586033,31.180915,31.180139,8617.58487,8612.853833,0.906035,254.569184,4.182402,81.571458,133.005552,8617.58487,21.828204,21.352291,23.659021,5.676976,0.665455
min,0.000000e+00,28000.000000,0.00000,0.000000,3.000000,664.000000,660.000000,1000.00000,950.000000,0.000000,14.770000,5.320000,0.000000,0.000000,1000.00000,0.000000,0.000000,0.000000,1.000000,0.000000
25%,4.700000e+04,76000.000000,0.00000,12.410000,13.805000,674.000000,670.000000,7775.00000,7750.000000,0.000000,244.242500,9.170000,624.000000,620.000000,7775.00000,16.000000,27.000000,51.000000,8.000000,0.000000
50%,6.500000e+04,99000.000000,0.00000,18.520000,17.940000,689.000000,685.000000,12000.00000,12000.000000,0.000000,369.520000,11.490000,689.000000,685.000000,12000.00000,31.000000,44.000000,67.000000,11.000000,0.000000
75%,9.400000e+04,132700.000000,0.00000,25.320000,23.477500,714.000000,710.000000,20000.00000,20000.000000,1.000000,572.850000,14.330000,734.000000,730.000000,20000.00000,50.000000,63.000000,81.000000,15.000000,0.000000
max,9.000000e+06,500000.000000,15.00000,999.000000,43.860000,850.000000,845.000000,35000.00000,35000.000000,5.000000,1354.660000,28.990000,850.000000,845.000000,35000.00000,152.000000,152.000000,120.000000,67.000000,23.000000


In [15]:
def add_feature(df):
    df['continuous_fico_diff_high'] = df['continuous_last_fico_range_high']-df['continuous_fico_range_high']
    df['continuous_fico_diff_low'] = df['continuous_last_fico_range_low']-df['continuous_fico_range_low']
    df['continuous_rec_ratio'] = df['continuous_pub_rec']/df['continuous_open_acc']
    return df

In [16]:
dftrain = add_feature(dftrain)
dftest = add_feature(dftest)

In [17]:
feature = list(dftrain.columns)
feature.remove('loan_status')
x_train, x_valid, y_train, y_valid = train_test_split(dftrain[feature], dftrain[label], test_size = 0.2, random_state = 42)

In [18]:
train_data = lgb.Dataset(x_train, label=y_train)
valid_data = lgb.Dataset(x_valid, label=y_valid)
test_data = lgb.Dataset(dftest[feature], label=dftest[label])

In [49]:
param = {'num_leaves': 31, 'max_depth':5,
         'objective': 'binary','learning_rate':0.1, 'metric': 'binary'}
num_round = 1000
bst2 = lgb.train(param, train_data, num_round, valid_sets=[valid_data],early_stopping_rounds=50)

[1]	valid_0's binary_logloss: 0.448371
Training until validation scores don't improve for 50 rounds.
[2]	valid_0's binary_logloss: 0.405765
[3]	valid_0's binary_logloss: 0.373175
[4]	valid_0's binary_logloss: 0.347038
[5]	valid_0's binary_logloss: 0.325588
[6]	valid_0's binary_logloss: 0.307744
[7]	valid_0's binary_logloss: 0.292795
[8]	valid_0's binary_logloss: 0.279998
[9]	valid_0's binary_logloss: 0.268991
[10]	valid_0's binary_logloss: 0.259619
[11]	valid_0's binary_logloss: 0.251632
[12]	valid_0's binary_logloss: 0.244693
[13]	valid_0's binary_logloss: 0.23859
[14]	valid_0's binary_logloss: 0.233331
[15]	valid_0's binary_logloss: 0.228781
[16]	valid_0's binary_logloss: 0.22473
[17]	valid_0's binary_logloss: 0.221208
[18]	valid_0's binary_logloss: 0.218261
[19]	valid_0's binary_logloss: 0.215616
[20]	valid_0's binary_logloss: 0.21333
[21]	valid_0's binary_logloss: 0.211357
[22]	valid_0's binary_logloss: 0.209694
[23]	valid_0's binary_logloss: 0.208168
[24]	valid_0's binary_logloss:

In [50]:
score_v2 = eval_model(bst2, dftest)
print('加特征后测试集acc比之前提升:',score_v2- origin_score)

训练集 {'binary_logloss': 0.1977088793757824}
测试集 0.91702
加特征后测试集acc比之前提升: 0.00045999999999990493


## stacking
第一层：多组不同参数的lgb

第二层：LR

In [22]:
from sklearn.linear_model import LogisticRegression

In [23]:
from sklearn.model_selection import KFold
kfold = KFold(n_splits=5)

In [54]:
class FitterBase(object):
    def __init__(self, label, metric, max_eval=100, opt=None):
        self.label = label
        self.metric = metric
        self.opt_params = dict()
        self.max_eval = max_eval
        self.opt = opt

    def get_loss(self, y, y_pred):
        if self.metric == 'error':
            return accuracy_score(y, y_pred)
        elif self.metric == 'precision':
            return 1 - precision_score(y, y_pred)
        elif self.metric == 'recall':
            return 1 - recall_score(y, y_pred)
        elif self.metric == 'logloss':
            return log_loss(y, y_pred)
        elif self.metric == 'macro_f1':
            return 1 - f1_score(y, y_pred, average='macro')
        elif self.metric == 'micro_f1':
            return 1 - f1_score(y, y_pred, average='micro')
        elif self.metric == 'auc':  # TODO: Add a warning checking if y_predict is all [0, 1], it should be probability
            return 1 - roc_auc_score(y, y_pred)
        else:
            raise Exception("Not implemented yet.")

In [61]:
class LGBFitter(FitterBase):
    def __init__(self, label='label', metric='error', opt=None, max_eval=100):
        super(LGBFitter, self).__init__(label, metric, max_eval)
        if opt is not None:
            self.opt = opt
        self.best_round = None
        self.clf = None

    def train_lgb(self, train_df, eval_df, params=None, use_best_eval=True):
        self.best_round = None
        dtrain = lgb.Dataset(train_df.drop(columns=[self.label]), train_df[self.label])
        deval = lgb.Dataset(eval_df.drop(columns=[self.label]), eval_df[self.label])
        evallist = [dtrain, deval]
        if params is None:
            use_params = deepcopy(self.opt_params)
        else:
            use_params = deepcopy(params)

        num_round = use_params.pop('num_round')
        if use_best_eval:
            with io.StringIO() as buf, redirect_stdout(buf):
                self.clf = lgb.train(use_params, dtrain, num_round, valid_sets=evallist)
                output = buf.getvalue().split("\n")
            min_error = np.inf
            min_index = 0
            for idx in range(len(output) - 1):
                if len(output[idx].split("\t")) == 3:
                    temp = float(output[idx].split("\t")[2].split(":")[1])
                    if min_error > temp:
                        min_error = temp
                        min_index = int(output[idx].split("\t")[0][1:-1])
            print("The minimum is attained in round %d" % (min_index + 1))
            self.best_round = min_index + 1
            return output
        else:
            with io.StringIO() as buf, redirect_stdout(buf):
                self.clf = lgb.train(use_params, dtrain, num_round, valid_sets=evallist)
                output = buf.getvalue().split("\n")
            self.best_round = num_round
            return output

    def train_k_fold(self, k_fold, train_data, test_data, params=None, drop_test_y=True, use_best_eval=True):
        acc_result = list()
        train_pred = np.empty(train_data.shape[0])
        test_pred = np.empty(test_data.shape[0])
        if drop_test_y:
            dtest = test_data.drop(columns=self.label)
        else:
            dtest = test_data

        models = list()
        for train_id, eval_id in k_fold.split(train_data):
            train_df = train_data.loc[train_id]
            eval_df = train_data.loc[eval_id]
            self.train_lgb(train_df, eval_df, params, use_best_eval)
            models.append(copy.deepcopy(self.clf))
            train_pred[eval_id] = self.clf.predict(eval_df.drop(columns=self.label), num_iteration=self.best_round)
            if self.metric == 'auc':
                y_pred = self.clf.predict(eval_df.drop(columns=[self.label]), num_iteration=self.best_round)
            else:
                y_pred = (self.clf.predict(eval_df.drop(columns=[self.label]),
                                           num_iteration=self.best_round) > 0.5).astype(int)
            acc_result.append(self.get_loss(eval_df[self.label], y_pred))
            test_pred += self.clf.predict(dtest, num_iteration=self.best_round)
        test_pred /= k_fold.n_splits
        return train_pred, test_pred, acc_result, models
    
    def train_stacking(self, kfold, train_data, test_data, param_list=None):
        model_num = len(param_list)
        # level1
        feature_train = np.empty((train_data.shape[0],model_num))
        feature_test = np.empty((test_data.shape[0],model_num))
        for index, params in enumerate(param_list):
            train_pred_L1, test_pred_L1, acc_result_L1, models_L1 = self.train_k_fold(kfold, dftrain, dftest, params = params)
            feature_train[:,index] = train_pred_L1
            feature_test[:,index] = test_pred_L1
        print('level1 acc,:',acc_result_L1)
        # level2
        clf_L2 = LogisticRegression(random_state=0).fit(feature_train, dftrain[self.label])
        test_pred_L2 = clf_L2.predict(feature_test)
        final_acc_result = self.get_loss(dftest[self.label], test_pred_L2)
        print('stacking acc:',final_acc_result)

In [62]:
fitter = LGBFitter(label='loan_status', max_eval=100, opt=None)

In [63]:
param_list = [
#     {'num_thread': 4, 'num_leaves': 12, 'learning_rate': 0.02, 'feature_fraction': 0.8, 'bagging_fraction': 0.8, 'metric': 'binary', 'objective': 'binary', 'num_round': 1000},
    {'num_thread': 4, 'num_leaves': 15, 'learning_rate': 0.1, 'feature_fraction': 0.5, 'bagging_fraction': 0.75, 'metric': 'binary', 'objective': 'binary', 'num_round': 1000},
    {'num_thread': 4, 'num_leaves': 12, 'learning_rate': 0.1, 'feature_fraction': 1, 'bagging_fraction': 0.8, 'extra_trees': True, 'metric': 'binary', 'objective': 'binary', 'num_round': 1000},
    {'num_thread': 4, 'num_leaves': 12, 'learning_rate': 0.1, 'feature_fraction': 0.9, 'bagging_fraction': 0.8, 'lambda_l1':0.1, 'metric': 'binary', 'objective': 'binary', 'num_round': 1000},
    {'num_thread': 4, 'num_leaves': 12, 'learning_rate': 0.1, 'feature_fraction': 0.6, 'bagging_fraction': 0.8, 'lambda_l2':0.1, 'metric': 'binary', 'objective': 'binary', 'num_round': 1000}
]

In [64]:
fitter.train_stacking(kfold, dftrain, dftest, param_list)

The minimum is attained in round 102
The minimum is attained in round 66
The minimum is attained in round 72
The minimum is attained in round 60
The minimum is attained in round 51
The minimum is attained in round 106
The minimum is attained in round 59
The minimum is attained in round 81
The minimum is attained in round 81
The minimum is attained in round 62
The minimum is attained in round 71
The minimum is attained in round 64
The minimum is attained in round 104
The minimum is attained in round 70
The minimum is attained in round 57
The minimum is attained in round 115
The minimum is attained in round 80
The minimum is attained in round 70
The minimum is attained in round 75
The minimum is attained in round 54
level1 acc,: [0.9267, 0.9183, 0.916, 0.9159, 0.9197]
stacking acc: 0.90986
